<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/mnist/Colab_Notebook_MNIST_Do_Label_Shift_Adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download predictions from zenodo

In [11]:
!wget https://zenodo.org/record/3458206/files/mnist_test_labels.txt.gz?download=1 -O test_labels.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-0.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-0.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-10.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-10.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-20.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-20.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-30.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-30.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-40.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-40.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-50.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-50.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-60.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-60.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-70.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-70.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-80.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-80.txt.gz
!wget https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-90.txt.gz?download=1 -O testpreacts_model_mnist_set-4000_seed-90.txt.gz
!wget https://zenodo.org/record/3458206/files/mnist_valid_labels.txt.gz?download=1 -O valid_labels.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-0.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-0.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-10.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-10.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-20.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-20.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-30.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-30.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-40.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-40.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-50.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-50.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-60.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-60.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-70.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-70.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-80.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-80.txt.gz
!wget https://zenodo.org/record/3458206/files/validpreacts_model_mnist_set-4000_seed-90.txt.gz?download=1 -O validpreacts_model_mnist_set-4000_seed-90.txt.gz

--2019-09-23 15:51:08--  https://zenodo.org/record/3458206/files/mnist_test_labels.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13746 (13K) [application/octet-stream]
Saving to: ‘test_labels.txt.gz’

test_labels.txt.gz  100%[===================>]  13.42K  --.-KB/s    in 0.1s    

2019-09-23 15:51:09 (115 KB/s) - ‘test_labels.txt.gz’ saved [13746/13746]

--2019-09-23 15:51:10--  https://zenodo.org/record/3458206/files/testpreacts_model_mnist_set-4000_seed-0.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 479104 (468K) [application/octet-stream]
Saving to: ‘testpreacts_model_mnist_set-4000_seed-0.txt.gz’

testpreacts_model_m 100%[===================>] 467.88K   999KB/s    in 0.5s    

2019-09-23 

In [12]:
!gunzip -f *.gz
!md5sum *.txt

b7cac25c27a1c23e8afea9173a7dc13e  test_labels.txt
8b3d5dc6b17c514a68f8cffe6415d278  testpreacts_model_mnist_set-16000_seed-0.txt
5b749df4cd3ea08a43789dabf05becc5  testpreacts_model_mnist_set-16000_seed-10.txt
b4225781e18cd714e5b4155f345e8491  testpreacts_model_mnist_set-16000_seed-20.txt
bcbde2b0701ca48bc33c8a5df95c5ce7  testpreacts_model_mnist_set-16000_seed-30.txt
74f547450bf2a477d4c54452c81f97e5  testpreacts_model_mnist_set-16000_seed-40.txt
c1511b9fe6dc30b1c13576e7c22ca658  testpreacts_model_mnist_set-16000_seed-50.txt
42fa4f1ce2e71ec3e2ec791baed3e833  testpreacts_model_mnist_set-16000_seed-60.txt
c49b0bb9553cec0d3ceb0eee36f01705  testpreacts_model_mnist_set-16000_seed-70.txt
63fb5b4fb6a0c45e5b25bbf296d86b1f  testpreacts_model_mnist_set-16000_seed-80.txt
08500b34a2bf3a2f7ea58d6e75825fdd  testpreacts_model_mnist_set-16000_seed-90.txt
de6c92ac961762a1771fa88a711d0849  testpreacts_model_mnist_set-4000_seed-0.txt
5901947a1d3f06f1b9586c6ef02cb583  testpreacts_model_mnist_set-4000_seed-1

In [3]:

![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd abstention
!git pull
!pip uninstall abstention
!pip install .
%cd ..

Cloning into 'abstention'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 330 (delta 72), reused 64 (delta 36), pack-reused 222
Receiving objects: 100% (330/330), 94.11 KiB | 4.28 MiB/s, done.
Resolving deltas: 100% (214/214), done.
/content/abstention
Already up to date.
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.3.0-cp36-none-any.whl size=46220 sha256=6e013d7744ed1c233d10df62cdb124daa8348da78c630678efbe75855d9c9f70
  Stored in directory: /tmp/pip-ephem-wheel-cache-usi1b9dd/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
/content


In [4]:
![[ -e label_shift_experiments ]] || git clone https://github.com/blindauth/labelshiftexperiments
%cd /content/labelshiftexperiments
!git pull
!pip uninstall labelshiftexperiments
!pip install .
%cd ..

Cloning into 'labelshiftexperiments'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 13), reused 24 (delta 8), pack-reused 0
Unpacking objects: 100% (32/32), done.
/content/labelshiftexperiments
Already up to date.
Processing /content/labelshiftexperiments
  Created wheel for labelshiftexperiments: filename=labelshiftexperiments-0.1.0.0-cp36-none-any.whl size=6273 sha256=89bbc7dfb79706da14b6344ccdf23b4b20f718bad90b00260c847f921a6730de
  Stored in directory: /tmp/pip-ephem-wheel-cache-0eijl06a/wheels/6d/56/a7/fc736645f87832f3f9af94ef6661744069f41cd66bab35ec1b
Successfully built labelshiftexperiments
/content


In [13]:
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
reload(abstention.figure_making_utils)
from abstention.calibration import (
    TempScaling, VectorScaling, NoBiasVectorScaling, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import gzip
import numpy as np
from collections import defaultdict, OrderedDict

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist
        
test_labels = cifarandmnist.read_preds(open("test_labels.txt"))
valid_labels = cifarandmnist.read_preds(open("valid_labels.txt"))

imbalanceadaptername_to_imbalanceadapter = {
    'em': EMImbalanceAdapter(),
    'bbse-hard': BBSEImbalanceAdapter(soft=False),
    'bbse-soft': BBSEImbalanceAdapter(soft=True)}

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('NBVS', NoBiasVectorScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions=[0,1,2,3,4,5,6,7,8,9])),
    ('VS', VectorScaling(verbose=False)),
    #('CVBCTS', CrossValidatedBCTS(verbose=True,
    #                              num_crossvalidation_splits=5,
    #                              betas_to_try=[0.0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10]
    #                             ))
])

adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'NBVS'),
    ('bbse-soft', 'BCTS'),
    ('bbse-soft', 'VS'),  
    #('bbse-soft', 'CVBCTS'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'NBVS'),
    ('em', 'BCTS'),
    ('em', 'VS'),
    #('em', 'CVBCTS'),
]


num_trials = 10
seeds = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
dirichlet_alphas_and_samplesize = [
    (0.1,2000), (1.0,2000),
    (0.1,4000), (1.0,4000),
    (0.1,8000), (1.0,8000)]
tweakone_alphas_and_samplesize = [
    (0.01,2000), (0.9,2000),
    (0.01,4000), (0.9,4000),
    (0.01,8000), (0.9,8000)]

print("Dirichlet shift")

(dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 dirichlet_alpha_to_samplesize_to_baselineacc,                        
 metric_to_samplesize_to_calibname_to_unshiftedvals) =\
  cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = dirichlet_alphas_and_samplesize,
    shifttype='dirichlet',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="validpreacts_model_mnist_set-4000_seed-",
    testglobprefix="testpreacts_model_mnist_set-4000_seed-",
    valid_labels=valid_labels,
    test_labels=test_labels)

print("Tweak one shift")

(tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 tweakone_alpha_to_samplesize_to_baselineacc,                        
 _) = cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = tweakone_alphas_and_samplesize,
    shifttype='tweakone',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="validpreacts_model_mnist_set-4000_seed-",
    testglobprefix="testpreacts_model_mnist_set-4000_seed-",
    valid_labels=valid_labels,
    test_labels=test_labels)



Dirichlet shift
Seed 0
Seed 10
Seed 20
Seed 30
Seed 40
Seed 50
Seed 60
Seed 70
Seed 80
Seed 90
Calibration stats
Metric ece
None 0.8572278936569507
TS 0.46577509216140484
NBVS 0.45883306650229577
BCTS 0.44442307606394915
VS 0.4469501459832477
Metric nll
None 0.09006560327913023
TS 0.08620000635152587
NBVS 0.0853903728784316
BCTS 0.08528682882235424
VS 0.08943416480554572
Metric jsdiv
None 0.0066070188292113655
TS 0.007404804879082741
NBVS 0.005714194386592998
BCTS 0.0063929340205031715
VS 0.0065972701982394055
On alpha 0.1 sample size 2000
Metric delta_acc
bbse-hard:None 0.019830000000000014 +/- 0.0011549458861782238
bbse-soft:None 0.01991500000000002 +/- 0.001168398173966947
bbse-soft:TS 0.020825000000000024 +/- 0.0012166433165048673
bbse-soft:NBVS 0.020790000000000017 +/- 0.001217137310381715
bbse-soft:BCTS 0.02094500000000002 +/- 0.001213307837727471
bbse-soft:VS 0.02037500000000002 +/- 0.001205466819338158
em:None 0.020725000000000025 +/- 0.0012020473906185778
em:TS 0.0215500000000

In [14]:
import json
import os
file_out = "label_shift_adaptation_results-4000.json"
dict_to_write = {
    "dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "dirichlet_alpha_to_samplesize_to_baselineacc":
     dirichlet_alpha_to_samplesize_to_baselineacc,
    "metric_to_samplesize_to_calibname_to_unshiftedvals":
     metric_to_samplesize_to_calibname_to_unshiftedvals,
    "tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "tweakone_alpha_to_samplesize_to_baselineacc":
     tweakone_alpha_to_samplesize_to_baselineacc
}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0

In [0]:
import gzip
import json
loaded_dicts = json.loads(gzip.open("label_shift_adaptation_results-4000.json.gz").read())
dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
dirichlet_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_baselineacc']
tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
tweakone_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_baselineacc']
metric_to_samplesize_to_calibname_to_unshiftedvals =\
    loaded_dicts['metric_to_samplesize_to_calibname_to_unshiftedvals']

In [16]:
from importlib import reload
import labelshiftexperiments
reload(labelshiftexperiments)
import labelshiftexperiments.maketable
reload (labelshiftexperiments.maketable)
from labelshiftexperiments.maketable import render_calibration_table

metricname_to_nicename = {'nll': 'nll', 'jsdiv': 'jsdiv', 'ece': 'ECE'}
calibname_to_nicename = {'None': "None", "TS": "TS",
                         "VS":"VS", "NBVS": "NBVS", "BCTS": "BCTS"}
  
from scipy.stats import norm
N = len(seeds)*num_trials
#Using the normal approximation at N=100;
# variance from https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
#Note that T = ((N+1)*N/2 - W)/2            
ustat_threshold = ((N*(N+1))/2 - norm.ppf(0.99)*np.sqrt(N*(N+1)*(2*N+1)/6.0))/2.0
  
print(render_calibration_table(
        metric_to_samplesize_to_calibname_to_unshiftedvals=
          metric_to_samplesize_to_calibname_to_unshiftedvals,
        #threshold of 8 comes from table https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
        #for one-tailed alpha=0.025 and n=10            
        ustat_threshold=ustat_threshold,
        metrics_in_table=['nll', 'jsdiv', 'ece'],
        samplesizes_in_table=['2000', '4000', '8000'],
        calibnames_in_table=['None', 'TS', 'NBVS', 'BCTS', 'VS'],
        metricname_to_nicename=metricname_to_nicename,
        calibname_to_nicename=calibname_to_nicename,
        caption="MNIST Calibration metric differences", label="mnistcalibrationcomparison",
        applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c c c | c c c | c c c }
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{nll} & \multicolumn{3}{| c}{jsdiv} & \multicolumn{3}{| c}{ECE}\\
    \cline{2-10}
    & $n$=2000 & $n$=4000 & $n$=8000 & $n$=2000 & $n$=4000 & $n$=8000 & $n$=2000 & $n$=4000 & $n$=8000\\
    \hline
    None & 0.09; 3.25 & 0.09; 3.77 & 0.09; 3.89 & 0.007; 1.74 & 0.007; 2.47 & 0.007; 2.72 & 0.857; 3.51 & 0.857; 3.61 & 0.857; 3.6\\
    TS & 0.086; 1.64 & 0.086; 2.43 & 0.086; 2.9 & 0.007; 3.06 & 0.007; 3.61 & 0.007; 3.87 & \textbf{0.466; 1.77} & 0.443; 1.84 & 0.42; 1.7\\
    NBVS & \textbf{0.085; 1.04} & 0.084; 0.95 & 0.083; 0.88 & \textbf{0.006; 0.92} & \textbf{0.005; 0.71} & \textbf{0.004; 0.31} & \textbf{0.459; 1.7} & 0.442; 1.76 & 0.434; 1.89\\
    BCTS & \textbf{0.085; 0.84} & \textbf{0.084; 0.58} & \textbf{0.083; 0.47} & 0.006; 1.9 & 0.005; 1.4 & 0.004; 1.31 & \

In [17]:
from labelshiftexperiments.maketable import render_adaptation_table
    
methodgroups = OrderedDict([
  ('em', ['em:None', 'em:TS', 'em:NBVS', 'em:BCTS', 'em:VS']),
  ('bbse', ['bbse-hard:None', 'bbse-soft:None',
            'bbse-soft:TS', 'bbse-soft:NBVS',
            'bbse-soft:BCTS', 'bbse-soft:VS'])])
samplesizes_in_table = ['2000', '4000', '8000']
adaptname_to_nicename = {'em': 'EM',
                           'bbse-soft': 'BBSE-soft',
                           'bbse-hard': 'BBSE-hard'}
calibname_to_nicename = {'None': 'None',
                         'TS': 'TS',
                         'NBVS': 'NBVS',
                         'BCTS': 'BCTS',
                         'VS': 'VS'}

dirichlet_alphas_in_table = ['0.1', '1.0']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="MNIST Metric: JS Divergence, dirichlet shift",
    label="mnistjsdivdirichletshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="MNIST Metric: $\\Delta$\\%Accuracy, dirichlet shift",
    label="mnistdeltaaccdirichletshift",
    applyunderline=False))

tweakone_alphas_in_table = ['0.01', '0.9']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="MNIST Metric: JS Divergence, tweakone shift",
    label="mnistjsdivtweakoneshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="MNIST Metric: $\\Delta$\\%Accuracy, tweakone shift",
    label="mnistdeltaacctweakoneshift",
    applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.1$} & \multicolumn{3}{| c}{$\alpha=1.0$}\\ 
    \cline{3-8}
    & & $n$=2000 & $n$=4000 & $n$=8000 & $n$=2000 & $n$=4000 & $n$=8000\\
    \hline
    \hline
    EM & None & 0.016; 2.89 & 0.015; 3.13 & 0.014; 3.12 & 0.01; 2.28 & 0.009; 2.6 & 0.008; 3.0\\
    EM & TS & \textbf{0.01; 1.75} & \textbf{0.009; 1.75} & \textbf{0.008; 2.02} & 0.009; 2.15 & 0.009; 2.78 & 0.008; 3.26\\
    EM & NBVS & \textbf{0.01; 1.66} & \textbf{0.009; 1.67} & \textbf{0.008; 1.72} & \textbf{0.009; 1.49} & \textbf{0.007; 1.33} & \textbf{0.005; 0.95}\\
    EM & BCTS & \textbf{0.01; 1.61} & \textbf{0.009; 1.57} & \textbf{0.008; 1.7} & \textbf{0.009; 1.73} & 0.007; 1.52 & 0.006; 1.41\\
    EM & VS & 0.0